## Factorize  
Assume $N = p.q$ and you know 2/3 least significant bits of $q$.  <br /> 
**Goal**: Find $p,q$. 

In [6]:
import math
from fpylll import *
import sympy
import  random
def rand_prime(bitsize):
    return sympy.randprime(2**(bitsize-1), 2**(bitsize))
                           
def base_primes(num_primes):
    primes = list(sympy.primerange(a=2, b=10 ** 5))
    return primes[:num_primes]

def int2bitstring(value, size = None):
    bits = "{:b}".format(value)
    if size:
        bits = '0'*(size-len(bits)) + bits
    return bits    
def M_sizes(matrix):
    res_matrix = IntegerMatrix(matrix)
    nrows = matrix.nrows
    ncols = matrix.ncols
    for i in range(nrows):
        for j in range(ncols):
            res_matrix[i, j] = len(int2bitstring(matrix[i, j]))
    return res_matrix
        

In [7]:
bit_size = 60
p = rand_prime(bit_size)
q = rand_prime(bit_size)
N = p*q
M = 2**((bit_size*2)//3+2)
q_known = q % M  # equivalent to p & ((1 << 30)- 1)

print(f"p=  {int2bitstring(p)}")
print(f"q=  {int2bitstring(q)}")
print(f"q_known_part= {int2bitstring(q_known, 20)}")

p=  100001010001000010111110001001010011001000110100100010101101
q=  101001100011101001101000100000100011111001011000001111000011
q_known_part= 101000100000100011111001011000001111000011


$$
\begin{align}
(xM+A)(yM+B) &= N\\
xyM^2 + (Ay+xB)M + AB &= N\\
Ay+xB &\equiv C \pmod M
\end{align}
$$
\begin{align}
k(Ay+xB) &\equiv kC &\pmod M  \\
y(kA)+x(kB) &\equiv kC &\pmod M\\
y(a)+x(b) &\equiv c &\pmod M
\end{align}

$$
L=\left(\begin{array}{ccc} 
A & B \\
M & 0\\
0 & M \\
\end{array}\right)
\rightarrow LLL \rightarrow
L_{red}\left(\begin{array}{ccc} 
a & b \\
* & *\\
* & * \\
\end{array}\right)\\
$$ 

In [8]:
A = p % M
B = q % M
x, y = p // M, q // M
C = ((N - A*B) // M)  % M
p_known = (N * pow(q_known, -1, M)) % M
assert p_known == A
assert q_known == B

In [9]:
print(f"p={int2bitstring(p)}")
print(f"A={int2bitstring(A)}, A={A}")
print(f"q={int2bitstring(q)}")
print(f"B={int2bitstring(B)}, B={B}")

L = IntegerMatrix(3, 2)
L[0, 0], L[0,1] = A, B
L[1, 0] = M
L[2, 1] = M
print(f"\nL_before=\n{L}")
LLL.reduction(L)
print(f"\nL_after=\n{L}")

p=100001010001000010111110001001010011001000110100100010101101
A=111110001001010011001000110100100010101101, A=4270592313517
q=101001100011101001101000100000100011111001011000001111000011
B=101000100000100011111001011000001111000011, B=2783741051843

L_before=
[ 4270592313517 2783741051843 ]
[ 4398046511104             0 ]
[             0 4398046511104 ]

L_after=
[       0       0 ]
[ 2157853  -68525 ]
[  792463 2012993 ]


## Diophantine equation

Now we transformed modular equation into diophantine equation:  
\begin{align}
ay+bx &\equiv c \pmod M\\
ay+bx &= c
\end{align}
Annoying part, just technique. 

 - Which non-zero row $(a, b)$ of $L_{red}$ should be taken? 
 - Can we do something if equality does not hold exactly? What if for small $d$ we have: $$ay + xb - c = dM.$$
 - How to find x, y? What if $gcd(a, b) \neq 1.$ 